Whitney Kenner
u0777962
HW6
CS 6017
7-17-23
Character Classification using CNNs with PyTorch


### Step 1: Data Acquisition + Cleanup

In [ ]:
import pandas as pd
df = pd.read_csv("fonts/TIMES.csv")
df

In [ ]:
def prepareDF(dataframe):
    df = dataframe.drop(columns=['font', 'fontVariant', 'strength', 'italic', 'orientation', 'm_top', 'm_left', 'originalH', 'originalW', 'h', 'w'])
    return df

In [ ]:
df = prepareDF(df)

In [ ]:
import numpy as np

def normalize_X_and_y(dataframe):
    
    df1 = dataframe[dataframe['m_label'] > 33]
    df = df1[df1['m_label'] < 126]
    yArray = np.array(df['m_label'])

    temp_dataframe = df.drop(columns=['m_label'])
    xArray = np.zeros((len(temp_dataframe), 20, 20))
    for row in range(0, temp_dataframe.shape[0]):
        for i in range(20):
            for j in range(20):
                string = 'r' + str(i) + 'c' + str(j)
                xArray[row, i, j] = temp_dataframe.iloc[row].loc[string] /255
    
    
    return xArray, yArray
        


In [ ]:
xVals, yVals = normalize_X_and_y(df)

In [ ]:
print(xVals[:2])

In [ ]:
def getAsciiDictionaries(yValues):
    unique_vals = set(yValues)
    setSize = len(unique_vals)
    index_to_ascii = {}
    ascii_to_index = {}
    index = 0
    for val in unique_vals:
        ascii_to_index[val] = index
        index_to_ascii[index] = val
        index+=1

    print(index_to_ascii)
    print(ascii_to_index)
    return ascii_to_index, index_to_ascii, setSize


In [ ]:
asciiToIndex, indexToAscii, setSize = getAsciiDictionaries(yVals)

In [ ]:
def convertToIndex(yValues, ascii_to_index):
    for i in range(len(yValues)):
        yValues[i] = ascii_to_index[yValues[i]]

    return yValues

In [ ]:
yVals = convertToIndex(yVals, asciiToIndex)

### Step 2: Build a PyTorch Network


We're going to use the PyTorch library, like we've seen in class, to build/train our network. Check out the notebooks we've made in class or the official documentation/tutorials.

To start with, we're going to use a model very similar to the MNIST CNN we used in class. It will consist of:

a Convolution2D layer with ReLU activations
a max pooling layer
another convolution layer
another max pooling layer
a dense layer with relu activation
a dense layer
Compile and train your network like we did in class. You'll probably have to use the np.reshape() function on your data to make PyTorch happy. I reshaped my X values like np.reshape(Xs, (-1, 1, 20, 20)) to get them in the right format.

For training, you'll want to check out torch.utils.data.DataLoader which can take a TensorDataset so you can iterate over batches like we did in class for the MNIST data.

In [ ]:
import torchvision
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader



In [ ]:
print(xVals.data.shape)
print(setSize)

In [ ]:
plt.figure( figsize= (10, 10) )

for ii in np.arange( 25 ):
    plt.subplot( 5, 5, ii+1 )
    plt.imshow( xVals[ii, :, :], cmap='Greys',interpolation='none' )

plt.show()

In [ ]:
Xs = np.reshape(xVals, (-1, 1, 20, 20))
Xs

In [ ]:
#split the data into train/test

print("Xs: ", len(Xs))
print("Ys: ", len(yVals))

x_tensor = torch.tensor(Xs, dtype=torch.float32)
y_tensor = torch.tensor(yVals)

print(len(x_tensor))
print(len(y_tensor))


x_train, x_test, y_train, y_test = train_test_split(x_tensor, y_tensor, random_state=1, test_size=0.9)

training_data = TensorDataset(x_train, y_train)
testing_data = TensorDataset(x_test, y_test)


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        c1Out = 6 # convolution layer 1 will output 6 "images": one for each filter it trains
        c2Out = 16 # similarly for the 2nd convolution layer
        self.conv1 = nn.Conv2d( 1, c1Out, 3 ) # 1-D input, c1Out outputs, filter size 3x3 pixels
        
        # (28-2) x (28 -2) x c1Out outputs  # "-2" because 3x3 mask loses the 1st/last row/column
        
        self.pool = nn.MaxPool2d( 2, 2 ) # down sample 2x2 blocks to 1 value
        
        # 13*13*c1Out
        
        self.conv2 = nn.Conv2d( c1Out, c2Out, 3 ) # Inputs comes from conv1, specify our #outputs, use 3x3 blocks again
        
        # (13-2)*(13 -2)*c2Out
        # pool again
        # (11/2)*(11/2)*c2Out = 5x5 x c2Out
        
        #this is tricky.  The convolutions each shave 1 pixel off around the border, and then the
        #max pools reduce the number of pixels by 4
        self.pooledOutputSize = c2Out * 3 * 3 # 16 outputs per image whose size has been reduced
        self.fc1 = nn.Linear( self.pooledOutputSize, 120 )
        self.fc2 = nn.Linear( 120, 84 )
        self.fc3 = nn.Linear( 84, setSize ) # 10 outputs at the end

    ################################################################################
    # Take an image (or images) and run it through all stages of the net:
    #    
    def forward( self, x ): # "batch" of images
        # x is 4D tensor:  (batch size, width, height, #channels (1, grayscale image))
        # after conv1:  (batch size, width adjusted, height adjusted, conv1 # outputs)
        # after max pool: (batch size, width/2, height/2, conv1 # outputs)
        
        # print(x.shape) # During creation / debugging, getting the shape of layers correct is challenging... so display them.
        #x = F.relu(self.conv1(x))
        x = self.conv1(x)
        # print(x.shape)
        x = F.relu(x)
        # print(x.shape)
        x = self.pool(x)
        # print(x.shape)

        # Split into 2 lines above
        #x = self.pool(F.relu(self.conv1(x)))  #apply convolution filter, then run it through relu activation function
        x = self.pool(F.relu(self.conv2(x))) #ditto
        #print(x.shape) #uncomment to see the size of this layer.  It helped me figure out what pooledOutputSize shoudl be

        # Flatten: turn the 5x5xc2Out array into a single 1xN array.  The dense layers expect a 1D thing
        x = x.view(-1, self.num_flat_features(x))
        # x = x.view(x.shape[0], -1)  #equivalent ways of reshaping the data to be 1D
        # x = x.view(batch_size( x.shape[0]) , -1)
        x = F.relu(self.fc1(x)) #apply dense layer 1
        x = F.relu(self.fc2(x)) #and dense layer 2, using ReLU activation
        x = self.fc3(x) #final dense layer.  No activation function on this
        return x
    
    #compute the output size after our convolution layers
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()

In [ ]:
def train( model, epochs, training_data ): # One epoch uses the entire training set (one batch at a time) - 60,000 images in this case
    
    criterion = nn.CrossEntropyLoss() # this is a way of measuring error (loss) for classification that takes the
                                      # "confidence" of a prediction into account.  High confidence, correct predictions are low cost, 
                                      # high confidence, wrong predictions are high cost, medium confidence predictions have cost

    # use the ADAM optimizer to find the best weights
    optimizer = optim.Adam( model.parameters(), lr= 1e-4 ) 
    
    #this loads data and gets it in the right format for us
    trainloader = torch.utils.data.DataLoader( training_data, batch_size=8,
                                               shuffle=True, num_workers=0 )

    for epoch in range( epochs ): # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate( trainloader, 0 ):
            # get the inputs; data is a list of [inputs, labels]

            inputs = x_tensor
            labels = y_tensor
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs) #predict the output with some training data
            loss = criterion(outputs, labels) #see how well we did

            loss.backward() #see how to change the weights to do better
            optimizer.step() #and actually change the weights

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

def evaluate( model, testing_data, iToA):  
    #load some test data
    testloader = torch.utils.data.DataLoader( testing_data, batch_size=1,
                                              shuffle=True, num_workers=0 )
    correct = 0
    total = 0

    with torch.no_grad(): # <- Since we are not training, the model does not need to calculate gradients
        count = 0
        for data in testloader:
            
            images, labels = data
            outputs = model( images )
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            if predicted != labels:
                ascii = chr(iToA[predicted.item()])

                plt.subplot(5,10, count+1)
                
                #plt.ylabel("actual: ", labels.item())
                plt.imshow(images[0,0])
                plt.title(ascii)
                plt.axis('off')
                
                count+=1
                if count >=50:
                    break
            #plt.subplot.figsize=(30,30)

    # Just do a coarse evaluation... how many did we predict correcly?
    print( 'Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total) )
    

In [ ]:
# Note: On my home (older PC), this takes 3-ish minutes to run...
print( "Training..." )
train( net, 15, training_data )


### Step 3: Exploration and Evaluation

In [ ]:

print( "Evaluating..." )
evaluate( net, testing_data, indexToAscii )

 What is its accuracy?  
 50% in this instance. I can also increase the number of epochs and it increases substantially

In [ ]:
#Create and train a different network topology (add more convolution layers, experiment with normalization (batch normalization or dropout), 
#explore other types/sizes of layer). Try to find a topology that works better than the one described above.
class NewNet(nn.Module):
    def __init__(self):
        super(NewNet, self).__init__()
        c1Out = 16 # convolution layer 1 will output 6 "images": one for each filter it trains
        c2Out = 32 # similarly for the 2nd convolution layer
        c3Out = 32
        self.conv1 = nn.Conv2d( 1, c1Out, 3 ) # 1-D input, c1Out outputs, filter size 3x3 pixels
        
        # (28-2) x (28 -2) x c1Out outputs  # "-2" because 3x3 mask loses the 1st/last row/column
        
        self.pool = nn.MaxPool2d( 2, 2 ) # down sample 2x2 blocks to 1 value
        
        # 13*13*c1Out
        
        self.conv2 = nn.Conv2d( c1Out, c2Out, 3 ) # Inputs comes from conv1, specify our #outputs, use 3x3 blocks again

        self.conv3 = nn.Conv3d(c2Out, c3Out, 3)
        
        # (13-2)*(13 -2)*c2Out
        # pool again
        # (11/2)*(11/2)*c2Out = 5x5 x c2Out
        
        #this is tricky.  The convolutions each shave 1 pixel off around the border, and then the
        #max pools reduce the number of pixels by 4
        self.pooledOutputSize = c2Out * 3 * 3 # 16 outputs per image whose size has been reduced
        self.fc1 = nn.Linear( self.pooledOutputSize, 120 )
        self.fc2 = nn.Linear( 120, 84 )
        self.fc3 = nn.Linear( 84, setSize ) # 10 outputs at the end

    ################################################################################
    # Take an image (or images) and run it through all stages of the net:
    #    
    def forward( self, x ): # "batch" of images
        # x is 4D tensor:  (batch size, width, height, #channels (1, grayscale image))
        # after conv1:  (batch size, width adjusted, height adjusted, conv1 # outputs)
        # after max pool: (batch size, width/2, height/2, conv1 # outputs)
        
        # print(x.shape) # During creation / debugging, getting the shape of layers correct is challenging... so display them.
        #x = F.relu(self.conv1(x))
        x = self.conv1(x)
        # print(x.shape)
        x = F.relu(x)
        # print(x.shape)
        x = self.pool(x)
        # print(x.shape)

        # Split into 2 lines above
        #x = self.pool(F.relu(self.conv1(x)))  #apply convolution filter, then run it through relu activation function
        x = self.pool(F.relu(self.conv2(x))) #ditto
        #print(x.shape) #uncomment to see the size of this layer.  It helped me figure out what pooledOutputSize shoudl be

        # Flatten: turn the 5x5xc2Out array into a single 1xN array.  The dense layers expect a 1D thing
        x = x.view(-1, self.num_flat_features(x))
        # x = x.view(x.shape[0], -1)  #equivalent ways of reshaping the data to be 1D
        # x = x.view(batch_size( x.shape[0]) , -1)
        x = F.relu(self.fc1(x)) #apply dense layer 1
        x = F.relu(self.fc2(x)) #and dense layer 2, using ReLU activation
        x = self.fc3(x) #final dense layer.  No activation function on this
        return x
    
    #compute the output size after our convolution layers
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


newnet = NewNet()


In [ ]:
# Note: On my home (older PC), this takes 3-ish minutes to run...
print( "Training..." )
train( newnet, 15, training_data)

In [ ]:
print( "Evaluating..." )
evaluate( newnet, testing_data, indexToAscii )

In [ ]:
#Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform?
df2 = pd.read_csv("fonts/ERAS.csv")
df2 = prepareDF(df2)
xValSitka, yValSitka = normalize_X_and_y(df2)

In [ ]:
ascii_to_index_sitka, index_to_ascii_sitka, setSize = getAsciiDictionaries(yValSitka)
Ys = convertToIndex(yValSitka, ascii_to_index_sitka)


plt.figure( figsize= (10, 10) )

for ii in np.arange( 25 ):
    plt.subplot( 5, 5, ii+1 )
    plt.imshow( xValSitka[ii, :, :], cmap='Greys',interpolation='none' )

plt.show()

Xs_sitka = np.reshape(xValSitka, (-1, 1, 20, 20))

x_tensor_sitka = torch.tensor(Xs_sitka, dtype=torch.float32)
y_tensor_sitka = torch.tensor(Ys)

print(len(x_tensor_sitka))
print(len(y_tensor_sitka))


x_train_sitka, x_test_sitka, y_train_sitka, y_test_sitka = train_test_split(x_tensor_sitka, y_tensor_sitka, random_state=1, test_size=0.9)

training_data_sitka = TensorDataset(x_train_sitka, y_train_sitka)
testing_data_sitka = TensorDataset(x_test_sitka, y_test_sitka)

In [ ]:
# # Note: On my home (older PC), this takes 3-ish minutes to run...
# print( "Training..." )
# train( net, 22, training_data_sitka )

In [ ]:
print( "Evaluating..." )
evaluate( net, testing_data_sitka, index_to_ascii_sitka )

How does it perform?
It performs terribly, somewhere between 1 and 7% when everything is re-run

In [ ]:
print( "Training..." )
train( net, 15, training_data_sitka)

In [ ]:
print( "Evaluating..." )
evaluate( net, testing_data_sitka, index_to_ascii_sitka)

How does your accuracy compare to the 1-font case? What accuracy do you see when testing with inputs from a font you didn't train on?  
it's much worse, the fonts I picked are quite different. my 2nd font also has a smaller sample size than the first one, so less training data  
most times I got 1% or 0% when testing with inputs I didn't train on, which makes sense

In [ ]:
#third font:

df_elephant = pd.read_csv("fonts/SERIF.csv")
df3 = prepareDF(df_elephant)
xValEl, yValEl = normalize_X_and_y(df3)

ascii_to_index_elephant, index_to_ascii_elephant, setSize = getAsciiDictionaries(yValEl)
Ys_el = convertToIndex(yValEl, ascii_to_index_elephant)


plt.figure( figsize= (10, 10) )

for ii in np.arange( 25 ):
    plt.subplot( 5, 5, ii+1 )
    plt.imshow( xValEl[ii, :, :], cmap='Greys',interpolation='none' )

plt.show()

Xs_el = np.reshape(xValEl, (-1, 1, 20, 20))

x_tensor_el = torch.tensor(Xs_el, dtype=torch.float32)
y_tensor_el = torch.tensor(Ys_el)

print(len(x_tensor_el))
print(len(y_tensor_el))


x_train_el, x_test_el, y_train_el, y_test_el = train_test_split(x_tensor_el, y_tensor_el, random_state=1, test_size=0.9)

training_data_el = TensorDataset(x_train_el, y_train_el)
testing_data_el = TensorDataset(x_test_el, y_test_el)


In [ ]:
print( "Evaluating..." )
evaluate( net, testing_data_el, index_to_ascii_elephant)

Do you notice any patterns? The network only produces the relative probabilities that the input is any of the possible characters. Can you find examples where the network is unsure of the result?  
my models seem to have a really hard time with expecting numbers and symbols. I removed all the unicode characters outside of the ascii range, so those are not part of the input, but it often seems to misclassify letters as numbers, I wonder if the sample size of numbers is smaller than other characters and therefor my model is bad at predicting them

### Step 4: Denoising

In [ ]:
#Next, we'll build and train a neural network - an autoencoder - for a different task: denoising images.
class CnnNet(nn.Module):
    def __init__(self):
        super(CnnNet, self).__init__()
        
        self.encodedSize = 32
        
        self.c1Out = 8 # filters from first conv layer
        self.c2Out = 8 # filters from 2nd conv layer
        
        #the padding here puts a "border" of 0s around the image, so that convolution layers don't "shrink" the image
        
        self.cv1 = nn.Conv2d(1, self.c1Out, 3, padding=1) #stick with 3x3 filters
        #output is 8x 28x28 images
        self.pool = nn.MaxPool2d(2,2)
        self.cv2 = nn.Conv2d(self.c1Out, self.c2Out, 3, padding=1)
        #reuse pool here
        
        self.downscaledSize = 20//4 #we add padding, so the conv2d layers don't change the size, just the max pools
        self.flattenedSize = self.downscaledSize*self.downscaledSize*self.c2Out
        
        self.fc1 = nn.Linear(self.flattenedSize, 64)
        self.fc2 = nn.Linear(64, self.encodedSize) #scale down to 64 features

        #now we're encoded, so go define decoding pieces
        
        self.fc3 = nn.Linear(self.encodedSize, 64) #scale down to 64 features
        self.fc4 = nn.Linear(64, self.flattenedSize)
        
        
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        # the padding is very important here so we don't have to guess a "frame" of pixels around the image
        self.cv3 = nn.Conv2d(self.c2Out, self.c1Out, 3, padding=1)
        # apply upsample again
        self.cv4 = nn.Conv2d(self.c1Out, 1, 3, padding=1)
        
        
    def compress(self, x):
        x = self.cv1(x)
        #print("shape after cv1", x.shape)
        x = F.relu(self.pool(x))
        #print("shape after pool1", x.shape)
        x = self.cv2(x)
        #print("after cv2", x.shape)
        x = F.relu(self.pool(x))
        #print("after pool 2", x.shape)
        x = x.view(-1, self.flattenedSize)
        #print("flattened shape", x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #now we have a low-d representation of our data.  If we were doing compression, we'd store this
        return x
    
    def decompress(self, x):
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        #print(x.shape)
        x = x.view(-1, self.c2Out, self.downscaledSize, self.downscaledSize)
        #print("unflattened shape", x.shape)
        x = self.upsample(x)
        #print("upsample", x.shape)
        x = F.relu(self.cv3(x))
        #print(x.shape, "after cv3")
        x = self.cv4(self.upsample(x))
        #print(x.shape, "after both upsamples")
        return x
    
    def forward(self, x):
        x = self.compress(x)
        x = self.decompress(x)
       
        return x

cnnNet = CnnNet() # treat these as just 28 D vectors


criterion = nn.MSELoss()

def trainCNN(model, epochs, train_data):
    # create an optimizer object
    # Adam optimizer with learning rate 1e-3
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, num_workers=0)
    
    for epoch in range(epochs):
        loss = 0
        
        running_loss = 0
        
        for i, data in enumerate(train_loader, 0):

            #same as yesterday, except we're not even looking at the labels!
            # since we're not using a CNN, we need to "flatten" the input images
            batch_features = data[0]
        
            # reset the gradients back to zero
            # PyTorch accumulates gradients on subsequent backward passes
            optimizer.zero_grad()
        
            # compute reconstructions
            outputs = model(batch_features)
            #print(batch_features.shape)
            #print(outputs.shape)
            # compute training reconstruction loss
            # again, same idea as yesterday, but we're measuring the error slightly differently
            # how well does the reconstructed image match the input image?
            train_loss = criterion(outputs, batch_features)
        
            # compute accumulated gradients
            train_loss.backward()
        
            # perform parameter update based on current gradients
            optimizer.step()
        
            # add the mini-batch training loss to epoch loss
            loss += train_loss.item()
    
            # print statistics
            running_loss += train_loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.8f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    
        # compute the epoch training loss
        loss = loss / len(train_loader)
    
        # display the epoch training loss
        print("epoch : {}/{}, loss = {:.8f}".format(epoch + 1, epochs, loss))

def evaluateCNN(model, test_data):
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True, num_workers=0)
    total_loss = 0
    with torch.no_grad():
        for data in test_loader:
            images = data[0]
            outputs = model(images)
            test_loss = criterion(outputs, images)
            total_loss += test_loss.item()

    print("overall loss: ", total_loss)



def drawComparisonsCNN(model, test_data):
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True, num_workers=0)
    plt.figure(figsize=(20, 25))
    for i, batch in enumerate(test_loader):
        if i >= 8: break
        images = batch[0]
        #print(images.shape)
        with torch.no_grad():
            reconstructed = model(images)
            for j in range(len(images)):
                #draw the original image
                ax = plt.subplot(32, 16, i*32 + j + 1)
                plt.imshow(images[j].reshape((20,20)), cmap="Greys", interpolation=None)
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
            
                #and the reconstructed version in the next row
                ax = plt.subplot(32, 16, i*32 + j + 1 + 16)
                plt.imshow(reconstructed[j].reshape((20,20)), cmap="Greys", interpolation=None)
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
        

In [ ]:

df_pre_sound = pd.read_csv("fonts/SERIF.csv")
df_pre_sound = prepareDF(df_pre_sound)
xValSound, yValSound = normalize_X_and_y(df_pre_sound)



ascii_to_index_sound, index_to_ascii_sound, setSize = getAsciiDictionaries(yValSound)
Ys = convertToIndex(yValSound, ascii_to_index_sound)


#add sound
noise_to_add = np.random.normal(0,.1,xValSound.shape)
xValSound = xValSound + noise_to_add


plt.figure( figsize= (10, 10) )

for ii in np.arange( 25 ):
    plt.subplot( 5, 5, ii+1 )
    plt.imshow( xValSound[ii, :, :], cmap='Greys',interpolation='none' )

plt.show()

Xs_sound = np.reshape(xValSound, (-1, 1, 20, 20))

x_tensor_sound = torch.tensor(Xs_sound, dtype=torch.float32)
y_tensor_sound = torch.tensor(Ys)

print(len(x_tensor_sound))
print(len(y_tensor_sound))


x_train_sound, x_test_sound, y_train_sound, y_test_sound = train_test_split(x_tensor_sound, y_tensor_sound, random_state=1, test_size=0.9)

training_data_sound = TensorDataset(x_train_sound, y_train_sound)
testing_data_sound = TensorDataset(x_test_sound, y_test_sound)

In [ ]:
trainCNN(cnnNet, 15, training_data_sound)


In [ ]:
evaluateCNN(cnnNet, testing_data_sound)

In [ ]:
#Create a plot showing the noisy and denoised versions of some inputs to verify that your denoiser had the desired effect.  Discuss your results.

drawComparisonsCNN(cnnNet, testing_data_sound)